In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
#df= pd.read_stata("bl_el_clean.dta")
df = pd.read_stata("new_cleaned het data.dta")

In [5]:
df.head(2)

,hhid,two_project_fulla_1,ls_full,ls_rest,ls_half,ls_poultry_full,ls_poultry_rest,ls_poultry_half,male_designee_full,male_designee_half,treated_HHs,share_all,precovid,own_ls_e,small_ls_e,large_ls_e,medium_ls_e,own_ls,small_ls_bl,medium_ls_bl,large_ls_bl,w05_ls_value_e,w05_ls_value,fattening_ls_count,sold_ls,sold_ls_count,consumed_ls,consumed_ls_count,ls_stable,dwelling_type_bin_1,dwelling_type_bin_2,dwelling_type_bin_3,dwelling_type_bin_4,dwelling_type_bin_5,dwelling_type_bin_6,dwelling_ownership_b,dwelling_ownershi_2,dwelling_rent,tertile_dwellperpers,tertile_dwellperp_3,tertile_dwellperp_4,water_source_dist,hh_share_latrine,latrine,water_source_bin_2,water_source_bin_3,water_heating_source,water_heating_sou_5,water_heating_sou_6,water_heating_sou_7,water_heating_sou_8,water_heating_sou_9,sewage_e_bin_1,sewage_e_bin_2,sewage_e_bin_3,sewage_e_bin_4,sewage_e_bin_5,sewage_e_bin_6,ceiling_e_bin_1,ceiling_e_bin_2,ceiling_e_bin_3,ceiling_e_bin_4,ceiling_e_bin_5,ceiling_e_bin_6,ceiling_e_bin_7,ceiling_e_bin_9,flooring_e_bin_1,flooring_e_bin_2,flooring_e_bin_3,flooring_e_bin_4,flooring_e_bin_5,lighting_e_bin_1,lighting_e_bin_2,lighting_e_bin_4,home_improve_src__10,home_improve_src__11,home_improve_src__12,home_improve_src__13,home_improve_src__14,home_improve_src__15,home_improve_amount,ratio_smart_mobil_16,cart_e,bicycle_e,vehicle_toktok_e,vehicle_tricycle_e,vehicle_motorcycle_e,has_fridge_e,has_tv_lcd_e,has_wash_fullyauto_e,has_heater_e,skip_meal_e,ratio_skiptimes_p_17,worry_about_food,got_medical_assist,medical_assist_place,medical_assist_pl_18,medical_assist_pl_19,medical_assist_pl_20,medical_assist_money,medical_assist_mo_21,medical_assist_mo_22,medical_assist_mo_23,ladder,ratio_eatoutside__24,non_food_imputed__25,nonfood_exp_mon_bl,food_exp_bl,nonagri_grp_proce_26,nonagri_grp_buy_sell,nonagri_grp_services,nonagri_atleast1_loc,nonagri_atleast1__27,nonagri_sumcapital,agri_own_yesno,agri_own_size_quir,agri_rent_yesno,agri_rent_size_quir,agri_rent_type_bin_2,agri_rent_type_bin_3,agri_rent_amount_bl,agri_rent_out_yesno,agri_rent_out_siz_28,agri_rent_out_type_b,agri_rent_out_typ_29,agri_rent_out_amount,agri_l_assets_cart,agri_l_assets_shovel,agri_l_assets_plow,agri_l_assets_sickle,agri_l_assets_other,agri_l_assets_non,hhmembers_agri_work,bl_fruit_trees_count,totalincome_witho_32,totalincome_1mo,femhd,oldhd,younghd,singhd,illithd,haschld,propdisabled_ill,hhdisabl_chld,hhdisabl_adult,can_start_busines_33,worked_with_busin_34,use_time_forrev_atl,use_time_rev_30da_35,use_time_foredu_atl,use_time_edu_30da_36,nobank_r_too_far_atl,nobank_r_too_expe_37,nobank_r_lack_of__38,nobank_r_no_trust_39,nobank_r_religiou_40,nobank_r_not_enou_41,nobank_r_some1_el_42,nobank_r_no_need_atl,rosca_r_nonfarm_e_43,rosca_r_for_marri_44,rosca_r_for_educa_45,rosca_r_buy_a_hou_46,rosca_r_medical_e_47,rosca_r_loss_of_a_48,rosca_r_other_atl,help_mother_bin_2_49,help_mother_bin_3_50,help_father_bin_2_51,help_father_bin_3_52,help_siblings_bin_53,help_siblings_bin_54,help_hhmembers_outsi,help_hhmembers_ou_55,help_friends_bin__56,help_friends_bin__57,help_others_bin_2_58,help_others_bin_3_59,health_cant_work__60,health_cant_work__61,dm_food_atleast,dm_cloth_atleast,dm_educ_atleast,dm_health_atleast,dm_home_atleast,dm_food_influence_bi,dm_food_influence_62,dm_food_influence_63,dm_cloth_influence_b,dm_cloth_influenc_64,dm_cloth_influenc_65,dm_educ_influence_bi,dm_educ_influence_66,dm_educ_influence_67,dm_health_influence_,dm_health_influen_68,dm_health_influen_69,dm_home_influence_bi,dm_home_influence_70,dm_home_influence_71,dm_money_earn_bin_72,dm_money_earn_bin_73,dm_money_earn_bin_74,dm_money_earn_bin_75,dm_spouse_earn_bi_76,dm_spouse_earn_bi_77,dm_spouse_earn_bi_78,dm_spouse_earn_bi_79,dm_daily_spend_bi_80,dm_daily_spend_bi_81,dm_daily_spend_bi_82,dm_daily_spend_bi_83,dm_major_spend_bi_84,dm_major_spend_bi_85,dm_major_spend_bi_86,dm_major_spend_bi_87,dm_fplan_child_bi_88,dm_fplan_child_bi_89,dm_fplan_child_bi_90,dm_fplan_child_bi_91,dm_child_educ_bin_92,dm_child_educ

In [6]:
# Chcecking all cols types
unique_dtypes = df.dtypes.map(lambda dt: dt.name).unique()
print(unique_dtypes)

['object' 'float32' 'category' 'int8' 'int16' 'int32' 'float64']


In [7]:
object_columns = df.select_dtypes(include=['object']).columns
object_columns 

Index(['hhid', 'hhid_id', 'agg_id', 'q_id'], dtype='object')

In [8]:
df.drop(columns=['hhid', 'hhid_id', 'agg_id', 'q_id','nn'], inplace=True)

In [9]:
object_columns = df.select_dtypes(include=['object']).columns
object_columns 

Index([], dtype='object')

---

In [10]:
df['own_value_ls'].isna().sum()

np.int64(0)

In [11]:
df['own_value_ls'].describe()

count      3206.000000
mean      10031.263672
std       21096.554688
min           0.000000
25%           0.000000
50%        1071.666626
75%        7000.000000
max      262300.000000
Name: own_value_ls, dtype: float64

In [12]:
#Removing the amout that we given to the treatment group from their reported values
df['own_value_ls'] = np.where(df['treatment'] == 1,df['own_value_ls'] - 11379.31, df['own_value_ls'] )


In [13]:
df['own_value_ls'].describe()

count      3206.000000
mean      10031.263672
std       21096.554688
min           0.000000
25%           0.000000
50%        1071.666626
75%        7000.000000
max      262300.000000
Name: own_value_ls, dtype: float64

In [14]:
df['treatment'].value_counts()

treatment
Control                  935
Full cost                793
Full cost, restricted    771
Half cost                707
Name: count, dtype: int64

---

# Transforming the column that will be used for CLAN:

from sklearn.preprocessing import StandardScaler

clan_cols = [
    "treated_HHs","share_all","small_ls_bl","medium_ls_bl","large_ls_bl","w05_ls_value",
    "fattening_ls_count","sold_ls_count","consumed_ls_count","dwelling_rent","water_source_dist",
    "hh_share_latrine","home_improve_amount","ratio_smart_mobil_16","ratio_skiptimes_p_17",
    "ratio_eatoutside__24","nonfood_exp_mon_bl","food_exp_bl","nonagri_sumcapital",
    "agri_own_size_quir","agri_rent_size_quir","agri_rent_amount_bl","agri_rent_out_siz_28",
    "agri_rent_out_amount","bl_fruit_trees_count","totalincome_witho_32","totalincome_1mo",
    "propdisabled_ill","use_time_rev_30da_35","use_time_edu_30da_36","health_cant_work__61",
    "genderrole_mean","hh_prop_prepplus","hh_prop_enrolled","hh_max_attain","hh_size",
    "hh_n_work","dep_ratio","hh_female_prop","own_value_ls","log_totalincome_1mo",
    "elset_sh_eq","inflated_ls_hh","tot_exp_2"
]

scaler = StandardScaler()
df[[f"{c}_std" for c in clan_cols]] = scaler.fit_transform(df[clan_cols]).copy()



---

## Cleaning the columns
Our dataset uses consistent suffixes: variables ending in _bl are baseline (pre-treatment), while those ending in _e are endline (post-treatment). To avoid post-treatment leakage when building features 𝑍, we’ll exclude all columns whose names end with _e and keep only _bl variables plus any unsuffixed fields we know are pre-treatment.

- Note, check with Dr. Ragui if we need to exclude other columns that can potentially casue leakage.

In [15]:
df = df.loc[:, ~df.columns.str.endswith('_e')].copy()

## Categorical Variables Cleaning

In [16]:
df['ladder'].value_counts()

ladder
1 - extremly poor living        1803
2                                420
5                                379
3                                376
4                                196
6                                 24
10 - very comfortable living       4
8                                  2
2.077067                           1
7                                  1
Name: count, dtype: int64

##### Fixing the column "ladder"

In [17]:
# 1) Drop the row(s) with ladder == 2.077067 
df = df[~df["ladder"].astype(str).str.strip().eq("2.077067")].copy()

# 2) Convert labels like "1 - extremly poor living" and "10 - very comfortable living" to integers
df["ladder"] = df["ladder"].astype(str).str.extract(r"^\s*(\d+)")[0].astype(int)


#### Converting the binary columns from string to int

In [18]:
yes_no_cols = ['own_ls','ls_stable','latrine','worry_about_food','got_medical_assist',
'agri_own_yesno', 'agri_rent_yesno', 'agri_rent_out_yesno', 'hhmembers_agri_work']

for col in yes_no_cols:
    df[col] = np.where(df[col] == 'Yes', 1, 0)


In [19]:
df.head(3)

,two_project_fulla_1,ls_full,ls_rest,ls_half,ls_poultry_full,ls_poultry_rest,ls_poultry_half,male_designee_full,male_designee_half,treated_HHs,share_all,precovid,own_ls,small_ls_bl,medium_ls_bl,large_ls_bl,w05_ls_value,fattening_ls_count,sold_ls,sold_ls_count,consumed_ls,consumed_ls_count,ls_stable,dwelling_type_bin_1,dwelling_type_bin_2,dwelling_type_bin_3,dwelling_type_bin_4,dwelling_type_bin_5,dwelling_type_bin_6,dwelling_ownership_b,dwelling_ownershi_2,dwelling_rent,tertile_dwellperpers,tertile_dwellperp_3,tertile_dwellperp_4,water_source_dist,hh_share_latrine,latrine,water_source_bin_2,water_source_bin_3,water_heating_source,water_heating_sou_5,water_heating_sou_6,water_heating_sou_7,water_heating_sou_8,water_heating_sou_9,sewage_e_bin_1,sewage_e_bin_2,sewage_e_bin_3,sewage_e_bin_4,sewage_e_bin_5,sewage_e_bin_6,ceiling_e_bin_1,ceiling_e_bin_2,ceiling_e_bin_3,ceiling_e_bin_4,ceiling_e_bin_5,ceiling_e_bin_6,ceiling_e_bin_7,ceiling_e_bin_9,flooring_e_bin_1,flooring_e_bin_2,flooring_e_bin_3,flooring_e_bin_4,flooring_e_bin_5,lighting_e_bin_1,lighting_e_bin_2,lighting_e_bin_4,home_improve_src__10,home_improve_src__11,home_improve_src__12,home_improve_src__13,home_improve_src__14,home_improve_src__15,home_improve_amount,ratio_smart_mobil_16,ratio_skiptimes_p_17,worry_about_food,got_medical_assist,medical_assist_place,medical_assist_pl_18,medical_assist_pl_19,medical_assist_pl_20,medical_assist_money,medical_assist_mo_21,medical_assist_mo_22,medical_assist_mo_23,ladder,ratio_eatoutside__24,non_food_imputed__25,nonfood_exp_mon_bl,food_exp_bl,nonagri_grp_proce_26,nonagri_grp_buy_sell,nonagri_grp_services,nonagri_atleast1_loc,nonagri_atleast1__27,nonagri_sumcapital,agri_own_yesno,agri_own_size_quir,agri_rent_yesno,agri_rent_size_quir,agri_rent_type_bin_2,agri_rent_type_bin_3,agri_rent_amount_bl,agri_rent_out_yesno,agri_rent_out_siz_28,agri_rent_out_type_b,agri_rent_out_typ_29,agri_rent_out_amount,agri_l_assets_cart,agri_l_assets_shovel,agri_l_assets_plow,agri_l_assets_sickle,agri_l_assets_other,agri_l_assets_non,hhmembers_agri_work,bl_fruit_trees_count,totalincome_witho_32,totalincome_1mo,femhd,oldhd,younghd,singhd,illithd,haschld,propdisabled_ill,hhdisabl_chld,hhdisabl_adult,can_start_busines_33,worked_with_busin_34,use_time_forrev_atl,use_time_rev_30da_35,use_time_foredu_atl,use_time_edu_30da_36,nobank_r_too_far_atl,nobank_r_too_expe_37,nobank_r_lack_of__38,nobank_r_no_trust_39,nobank_r_religiou_40,nobank_r_not_enou_41,nobank_r_some1_el_42,nobank_r_no_need_atl,rosca_r_nonfarm_e_43,rosca_r_for_marri_44,rosca_r_for_educa_45,rosca_r_buy_a_hou_46,rosca_r_medical_e_47,rosca_r_loss_of_a_48,rosca_r_other_atl,help_mother_bin_2_49,help_mother_bin_3_50,help_father_bin_2_51,help_father_bin_3_52,help_siblings_bin_53,help_siblings_bin_54,help_hhmembers_outsi,help_hhmembers_ou_55,help_friends_bin__56,help_friends_bin__57,help_others_bin_2_58,help_others_bin_3_59,health_cant_work__60,health_cant_work__61,dm_food_atleast,dm_cloth_atleast,dm_educ_atleast,dm_health_atleast,dm_home_atleast,dm_food_influence_bi,dm_food_influence_62,dm_food_influence_63,dm_cloth_influence_b,dm_cloth_influenc_64,dm_cloth_influenc_65,dm_educ_influence_bi,dm_educ_influence_66,dm_educ_influence_67,dm_health_influence_,dm_health_influen_68,dm_health_influen_69,dm_home_influence_bi,dm_home_influence_70,dm_home_influence_71,dm_money_earn_bin_72,dm_money_earn_bin_73,dm_money_earn_bin_74,dm_money_earn_bin_75,dm_spouse_earn_bi_76,dm_spouse_earn_bi_77,dm_spouse_earn_bi_78,dm_spouse_earn_bi_79,dm_daily_spend_bi_80,dm_daily_spend_bi_81,dm_daily_spend_bi_82,dm_daily_spend_bi_83,dm_major_spend_bi_84,dm_major_spend_bi_85,dm_major_spend_bi_86,dm_major_spend_bi_87,dm_fplan_child_bi_88,dm_fplan_child_bi_89,dm_fplan_child_bi_90,dm_fplan_child_bi_91,dm_child_educ_bin_92,dm_child_educ_bin_93,dm_child_educ_bin_94,dm_child_educ_bin_95,dm_business_bin_1_96,dm_business_bin_2_97,dm_business_bin_3_98,dm_business_bin_4_99,health_rank_bin_1_100,health_rank_bin_2_101,health_rank_bin_3_102,health_ra

In [20]:
print(df['haschld'].value_counts())
print()
print(df['illithd'].value_counts())
print()
print(df['hhdisabl_chld'].value_counts())
print()
print(df['hhdisabl_adult'].value_counts())


haschld
HH has child    2929
0.0              277
Name: count, dtype: int64

illithd
illiterate HH-head    1898
0.0                   1308
Name: count, dtype: int64

hhdisabl_chld
0.0                      2434
HH has disabled child     772
Name: count, dtype: int64

hhdisabl_adult
HH has disabled adult    1657
0.0                      1549
Name: count, dtype: int64


In [21]:
df['femhd'].value_counts()

femhd
o.w               2799
female HH-head     407
Name: count, dtype: int64

In [22]:
binary_string_cols = ['haschld', 'illithd', 'hhdisabl_chld', 'hhdisabl_adult','oldhd','younghd','singhd']

for col in binary_string_cols:
    df[col] = (pd.to_numeric(df[col], errors="coerce") != 0).astype("int8")

df['femhd'] = np.where(df['femhd'] == 'female HH-head', 1, 0)


In [23]:
print(df['haschld'].value_counts())
print()
print(df['illithd'].value_counts())
print()
print(df['hhdisabl_chld'].value_counts())
print()
print(df['hhdisabl_adult'].value_counts())
print()
print(df['oldhd'].value_counts())
print()
print(df['younghd'].value_counts())
print()
print(df['singhd'].value_counts())
print()
print(df['femhd'].value_counts())

haschld
1    2929
0     277
Name: count, dtype: int64

illithd
1    1898
0    1308
Name: count, dtype: int64

hhdisabl_chld
0    2434
1     772
Name: count, dtype: int64

hhdisabl_adult
1    1657
0    1549
Name: count, dtype: int64

oldhd
0    3067
1     139
Name: count, dtype: int64

younghd
0    3162
1      44
Name: count, dtype: int64

singhd
0    2878
1     328
Name: count, dtype: int64

femhd
0    2799
1     407
Name: count, dtype: int64


### Clean some binary columns to int instead of float

#### Ask about this variable:

In [24]:
df['inflated_ls_hh'].value_counts()

inflated_ls_hh
3.793103    1564
0.000000     935
1.896552     707
Name: count, dtype: int64

In [25]:
# columns with exactly two unique values
binary_cols = [c for c in df.columns if df[c].nunique() == 2]

for col in binary_cols:
    df[col] = df[col].astype("Int64")

### **Creating the Full Cost vs Control DataFrame**

In [26]:
full = df[(df['treatment'] == 'Control') | (df['treatment'] == 'Full cost')].copy() # Filtering to only include full cost and control

full['D'] = np.where(full['treatment'] == 'Full cost',1,0) # D=1 is for full cost, D=0 is for control

full.drop(columns=['treatment'], inplace=True)

full.shape[0]

1728

In [27]:
full.head(3)

,two_project_fulla_1,ls_full,ls_rest,ls_half,ls_poultry_full,ls_poultry_rest,ls_poultry_half,male_designee_full,male_designee_half,treated_HHs,share_all,precovid,own_ls,small_ls_bl,medium_ls_bl,large_ls_bl,w05_ls_value,fattening_ls_count,sold_ls,sold_ls_count,consumed_ls,consumed_ls_count,ls_stable,dwelling_type_bin_1,dwelling_type_bin_2,dwelling_type_bin_3,dwelling_type_bin_4,dwelling_type_bin_5,dwelling_type_bin_6,dwelling_ownership_b,dwelling_ownershi_2,dwelling_rent,tertile_dwellperpers,tertile_dwellperp_3,tertile_dwellperp_4,water_source_dist,hh_share_latrine,latrine,water_source_bin_2,water_source_bin_3,water_heating_source,water_heating_sou_5,water_heating_sou_6,water_heating_sou_7,water_heating_sou_8,water_heating_sou_9,sewage_e_bin_1,sewage_e_bin_2,sewage_e_bin_3,sewage_e_bin_4,sewage_e_bin_5,sewage_e_bin_6,ceiling_e_bin_1,ceiling_e_bin_2,ceiling_e_bin_3,ceiling_e_bin_4,ceiling_e_bin_5,ceiling_e_bin_6,ceiling_e_bin_7,ceiling_e_bin_9,flooring_e_bin_1,flooring_e_bin_2,flooring_e_bin_3,flooring_e_bin_4,flooring_e_bin_5,lighting_e_bin_1,lighting_e_bin_2,lighting_e_bin_4,home_improve_src__10,home_improve_src__11,home_improve_src__12,home_improve_src__13,home_improve_src__14,home_improve_src__15,home_improve_amount,ratio_smart_mobil_16,ratio_skiptimes_p_17,worry_about_food,got_medical_assist,medical_assist_place,medical_assist_pl_18,medical_assist_pl_19,medical_assist_pl_20,medical_assist_money,medical_assist_mo_21,medical_assist_mo_22,medical_assist_mo_23,ladder,ratio_eatoutside__24,non_food_imputed__25,nonfood_exp_mon_bl,food_exp_bl,nonagri_grp_proce_26,nonagri_grp_buy_sell,nonagri_grp_services,nonagri_atleast1_loc,nonagri_atleast1__27,nonagri_sumcapital,agri_own_yesno,agri_own_size_quir,agri_rent_yesno,agri_rent_size_quir,agri_rent_type_bin_2,agri_rent_type_bin_3,agri_rent_amount_bl,agri_rent_out_yesno,agri_rent_out_siz_28,agri_rent_out_type_b,agri_rent_out_typ_29,agri_rent_out_amount,agri_l_assets_cart,agri_l_assets_shovel,agri_l_assets_plow,agri_l_assets_sickle,agri_l_assets_other,agri_l_assets_non,hhmembers_agri_work,bl_fruit_trees_count,totalincome_witho_32,totalincome_1mo,femhd,oldhd,younghd,singhd,illithd,haschld,propdisabled_ill,hhdisabl_chld,hhdisabl_adult,can_start_busines_33,worked_with_busin_34,use_time_forrev_atl,use_time_rev_30da_35,use_time_foredu_atl,use_time_edu_30da_36,nobank_r_too_far_atl,nobank_r_too_expe_37,nobank_r_lack_of__38,nobank_r_no_trust_39,nobank_r_religiou_40,nobank_r_not_enou_41,nobank_r_some1_el_42,nobank_r_no_need_atl,rosca_r_nonfarm_e_43,rosca_r_for_marri_44,rosca_r_for_educa_45,rosca_r_buy_a_hou_46,rosca_r_medical_e_47,rosca_r_loss_of_a_48,rosca_r_other_atl,help_mother_bin_2_49,help_mother_bin_3_50,help_father_bin_2_51,help_father_bin_3_52,help_siblings_bin_53,help_siblings_bin_54,help_hhmembers_outsi,help_hhmembers_ou_55,help_friends_bin__56,help_friends_bin__57,help_others_bin_2_58,help_others_bin_3_59,health_cant_work__60,health_cant_work__61,dm_food_atleast,dm_cloth_atleast,dm_educ_atleast,dm_health_atleast,dm_home_atleast,dm_food_influence_bi,dm_food_influence_62,dm_food_influence_63,dm_cloth_influence_b,dm_cloth_influenc_64,dm_cloth_influenc_65,dm_educ_influence_bi,dm_educ_influence_66,dm_educ_influence_67,dm_health_influence_,dm_health_influen_68,dm_health_influen_69,dm_home_influence_bi,dm_home_influence_70,dm_home_influence_71,dm_money_earn_bin_72,dm_money_earn_bin_73,dm_money_earn_bin_74,dm_money_earn_bin_75,dm_spouse_earn_bi_76,dm_spouse_earn_bi_77,dm_spouse_earn_bi_78,dm_spouse_earn_bi_79,dm_daily_spend_bi_80,dm_daily_spend_bi_81,dm_daily_spend_bi_82,dm_daily_spend_bi_83,dm_major_spend_bi_84,dm_major_spend_bi_85,dm_major_spend_bi_86,dm_major_spend_bi_87,dm_fplan_child_bi_88,dm_fplan_child_bi_89,dm_fplan_child_bi_90,dm_fplan_child_bi_91,dm_child_educ_bin_92,dm_child_educ_bin_93,dm_child_educ_bin_94,dm_child_educ_bin_95,dm_business_bin_1_96,dm_business_bin_2_97,dm_business_bin_3_98,dm_business_bin_4_99,health_rank_bin_1_100,health_rank_bin_2_101,health_rank_bin_3_102,health_ra

---

## Choosing p(Z)



##### **p(Z) is the probability of being assigned to a treatment group.**

We chosse geographical clusters that each have 5 households. For each cluster that has 5, 1 was randomly choosen as treatment "وتم الاختيار العشوائي لتجمع واحد من بين الخماسية لكل مجموعة من التدخلات ". and by 1 i mean 1 full treatment- full cost, 1 for treatment_full cost restricted, and 1 for half cost. All choosen at random from the five groups, the two left are the control (and in somecases there was only one control). 

**How we should set \(p(Z)\) given 3 treatment arms and 2 control clusters (per group-of-five)**

- We choose a **single-arm vs Control** contrast (e.g., *Full cost* vs Control).
- Define the indicator:
  - \(D=1\) if the unit is in **that one treatment arm** (e.g., *Full cost*),
  - \(D=0\) if the unit is in **Control**,
  - and we **exclude** the other two treatment arms (or equivalently, we condition on being in \(\{\text{that arm}, \text{Control}\}\)).

- In a **group of five** (“quintet”), there are **1** cluster for that arm and **2** clusters for Control in this contrast.  
  Therefore,

$$
p(Z) \;\equiv\; \Pr\!\big(D=1 \,\big|\, D \in \{\text{Full cost},\ \text{Control}\},\, Z\big)
\;=\; \frac{1}{1+2} \;=\; \tfrac{1}{3}.
$$


- If instead we pooled **any treatment** (all three arms) vs Control and coded \(D=1\) for “any treatment,” then in a group-of-five there are **3** treated clusters vs **2** control clusters:
$$
p(Z)\;\equiv\;\Pr\!\big(D=1 \,\big|\, D\!\in\!\{\text{any treatment},\ \text{Control}\},\ \text{group of five (3 treated, 2 control)}\big)
\;=\;\frac{3}{3+2}\;=\;\frac{3}{5}\;=\;0.6.
$$

- **Our main specification:** single-arm vs Control ⇒ use \(p(Z)=\tfrac{1}{3}\) for everyone (given the data lack the 4-vs-5 flag).
- **Sensitivity check:** to cover the occasional **group-of-four** (“quartet,” 1 arm vs 1 control), re-run once with
  $$
  p(Z)\;=\;\tfrac{1}{2}.
  $$


In [28]:
p_of_z = 1/3

---

# **Splitting**
- The data is splitted into 50% main (train) and 50% aux (test)
- The split is stratified by treatment, meaning that for each group (control and treatment), 50% of the data is used for training and 50% for testing.

In [29]:
# D-stratified 50/50 split → persistent 'fold' column

# first assign all values to main
full["fold"] = "main" 

# Then assign half of each group of D (0 & 1) to aux randomly.
aux_idx = (full.groupby("D", group_keys=False)
               .apply(lambda g: g.sample(frac=0.5, random_state=123))).index   # group_keys=False
full.loc[aux_idx, "fold"] = "aux"

C:\Users\ahmed\AppData\Local\Temp\ipykernel_22692\4114330533.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(frac=0.5, random_state=123))).index   # group_keys=False


In [30]:
full["fold"].value_counts()

fold
aux     864
main    864
Name: count, dtype: int64

---
---
---

## ML proxy:

In [31]:
from xgboost import XGBRegressor

### First training the two models on the "aux" dataset

In [32]:
# Defining the data for the two ML models:
u_0_data = full[(full['fold'] == 'aux') & (full['D'] == 0)].copy()
u_1_data = full[(full['fold'] == 'aux') & (full['D'] == 1)].copy()

# 1) Define X, y for each aux subset

drop_cols = ["own_value_ls", "D", "fold"]

X_0 = u_0_data.drop(columns=drop_cols)
y_0 = u_0_data["own_value_ls"]

X_1 = u_1_data.drop(columns=drop_cols)
y_1 = u_1_data["own_value_ls"]


### Trying CatBoost

#### First getting list of col names of the cat cols

In [33]:
full.head(3)

,two_project_fulla_1,ls_full,ls_rest,ls_half,ls_poultry_full,ls_poultry_rest,ls_poultry_half,male_designee_full,male_designee_half,treated_HHs,share_all,precovid,own_ls,small_ls_bl,medium_ls_bl,large_ls_bl,w05_ls_value,fattening_ls_count,sold_ls,sold_ls_count,consumed_ls,consumed_ls_count,ls_stable,dwelling_type_bin_1,dwelling_type_bin_2,dwelling_type_bin_3,dwelling_type_bin_4,dwelling_type_bin_5,dwelling_type_bin_6,dwelling_ownership_b,dwelling_ownershi_2,dwelling_rent,tertile_dwellperpers,tertile_dwellperp_3,tertile_dwellperp_4,water_source_dist,hh_share_latrine,latrine,water_source_bin_2,water_source_bin_3,water_heating_source,water_heating_sou_5,water_heating_sou_6,water_heating_sou_7,water_heating_sou_8,water_heating_sou_9,sewage_e_bin_1,sewage_e_bin_2,sewage_e_bin_3,sewage_e_bin_4,sewage_e_bin_5,sewage_e_bin_6,ceiling_e_bin_1,ceiling_e_bin_2,ceiling_e_bin_3,ceiling_e_bin_4,ceiling_e_bin_5,ceiling_e_bin_6,ceiling_e_bin_7,ceiling_e_bin_9,flooring_e_bin_1,flooring_e_bin_2,flooring_e_bin_3,flooring_e_bin_4,flooring_e_bin_5,lighting_e_bin_1,lighting_e_bin_2,lighting_e_bin_4,home_improve_src__10,home_improve_src__11,home_improve_src__12,home_improve_src__13,home_improve_src__14,home_improve_src__15,home_improve_amount,ratio_smart_mobil_16,ratio_skiptimes_p_17,worry_about_food,got_medical_assist,medical_assist_place,medical_assist_pl_18,medical_assist_pl_19,medical_assist_pl_20,medical_assist_money,medical_assist_mo_21,medical_assist_mo_22,medical_assist_mo_23,ladder,ratio_eatoutside__24,non_food_imputed__25,nonfood_exp_mon_bl,food_exp_bl,nonagri_grp_proce_26,nonagri_grp_buy_sell,nonagri_grp_services,nonagri_atleast1_loc,nonagri_atleast1__27,nonagri_sumcapital,agri_own_yesno,agri_own_size_quir,agri_rent_yesno,agri_rent_size_quir,agri_rent_type_bin_2,agri_rent_type_bin_3,agri_rent_amount_bl,agri_rent_out_yesno,agri_rent_out_siz_28,agri_rent_out_type_b,agri_rent_out_typ_29,agri_rent_out_amount,agri_l_assets_cart,agri_l_assets_shovel,agri_l_assets_plow,agri_l_assets_sickle,agri_l_assets_other,agri_l_assets_non,hhmembers_agri_work,bl_fruit_trees_count,totalincome_witho_32,totalincome_1mo,femhd,oldhd,younghd,singhd,illithd,haschld,propdisabled_ill,hhdisabl_chld,hhdisabl_adult,can_start_busines_33,worked_with_busin_34,use_time_forrev_atl,use_time_rev_30da_35,use_time_foredu_atl,use_time_edu_30da_36,nobank_r_too_far_atl,nobank_r_too_expe_37,nobank_r_lack_of__38,nobank_r_no_trust_39,nobank_r_religiou_40,nobank_r_not_enou_41,nobank_r_some1_el_42,nobank_r_no_need_atl,rosca_r_nonfarm_e_43,rosca_r_for_marri_44,rosca_r_for_educa_45,rosca_r_buy_a_hou_46,rosca_r_medical_e_47,rosca_r_loss_of_a_48,rosca_r_other_atl,help_mother_bin_2_49,help_mother_bin_3_50,help_father_bin_2_51,help_father_bin_3_52,help_siblings_bin_53,help_siblings_bin_54,help_hhmembers_outsi,help_hhmembers_ou_55,help_friends_bin__56,help_friends_bin__57,help_others_bin_2_58,help_others_bin_3_59,health_cant_work__60,health_cant_work__61,dm_food_atleast,dm_cloth_atleast,dm_educ_atleast,dm_health_atleast,dm_home_atleast,dm_food_influence_bi,dm_food_influence_62,dm_food_influence_63,dm_cloth_influence_b,dm_cloth_influenc_64,dm_cloth_influenc_65,dm_educ_influence_bi,dm_educ_influence_66,dm_educ_influence_67,dm_health_influence_,dm_health_influen_68,dm_health_influen_69,dm_home_influence_bi,dm_home_influence_70,dm_home_influence_71,dm_money_earn_bin_72,dm_money_earn_bin_73,dm_money_earn_bin_74,dm_money_earn_bin_75,dm_spouse_earn_bi_76,dm_spouse_earn_bi_77,dm_spouse_earn_bi_78,dm_spouse_earn_bi_79,dm_daily_spend_bi_80,dm_daily_spend_bi_81,dm_daily_spend_bi_82,dm_daily_spend_bi_83,dm_major_spend_bi_84,dm_major_spend_bi_85,dm_major_spend_bi_86,dm_major_spend_bi_87,dm_fplan_child_bi_88,dm_fplan_child_bi_89,dm_fplan_child_bi_90,dm_fplan_child_bi_91,dm_child_educ_bin_92,dm_child_educ_bin_93,dm_child_educ_bin_94,dm_child_educ_bin_95,dm_business_bin_1_96,dm_business_bin_2_97,dm_business_bin_3_98,dm_business_bin_4_99,health_rank_bin_1_100,health_rank_bin_2_101,health_rank_bin_3_102,health_ra

In [34]:
binary_cols 

['two_project_fulla_1',
 'ls_full',
 'ls_rest',
 'ls_half',
 'ls_poultry_full',
 'ls_poultry_rest',
 'ls_poultry_half',
 'male_designee_full',
 'male_designee_half',
 'precovid',
 'own_ls',
 'sold_ls',
 'consumed_ls',
 'ls_stable',
 'dwelling_type_bin_1',
 'dwelling_type_bin_2',
 'dwelling_type_bin_3',
 'dwelling_type_bin_4',
 'dwelling_type_bin_5',
 'dwelling_type_bin_6',
 'dwelling_ownership_b',
 'dwelling_ownershi_2',
 'tertile_dwellperpers',
 'tertile_dwellperp_3',
 'tertile_dwellperp_4',
 'latrine',
 'water_source_bin_2',
 'water_source_bin_3',
 'water_heating_source',
 'water_heating_sou_5',
 'water_heating_sou_6',
 'water_heating_sou_7',
 'water_heating_sou_8',
 'water_heating_sou_9',
 'sewage_e_bin_1',
 'sewage_e_bin_2',
 'sewage_e_bin_3',
 'sewage_e_bin_4',
 'sewage_e_bin_5',
 'sewage_e_bin_6',
 'ceiling_e_bin_1',
 'ceiling_e_bin_2',
 'ceiling_e_bin_3',
 'ceiling_e_bin_4',
 'ceiling_e_bin_5',
 'ceiling_e_bin_6',
 'ceiling_e_bin_7',
 'ceiling_e_bin_9',
 'flooring_e_bin_1',
 'fl

from catboost import CatBoostRegressor, Pool

# catboost params (analogous to your xgb_params)
cb_params = dict(
    iterations=300,
    depth=9,
    learning_rate=0.05,
    subsample=0.8,
    rsm=0.8,
    bootstrap_type="Bernoulli",
    loss_function="RMSE",
    eval_metric="RMSE",
    random_seed=39,
    thread_count=-1,
    verbose=False,
    allow_writing_files=False,
)

# fit
model_u1 = CatBoostRegressor(**cb_params).fit(Pool(X_1, y_1, cat_features=binary_cols))
model_u0 = CatBoostRegressor(**cb_params).fit(Pool(X_0, y_0, cat_features=binary_cols))

### Trying XGBoost (default for now)

In [35]:
# 2) Same model type/params for both fits 
xgb_params = dict(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=39,
    n_jobs=-1,
    objective="reg:squarederror",
    enable_categorical=True
)

model_u1 = XGBRegressor(**xgb_params).fit(X_1, y_1)  # μ̂1(Z)
model_u0 = XGBRegressor(**xgb_params).fit(X_0, y_0)  # μ̂0(Z)

### Then, after training the two models, we can use them to predict the treatment effect on the "main" dataset
- This is done by first filtering the "full" dataset to include only the main rows
- Then, we use the two models to predict the treatment effects for each row in the "main" dataset.
- Last, we calculate the treatment effect for each row in the "main" dataset by subtracting the predicted treatment effect from the actual treatment effect.

> We then add the ML proxy a new column in the "full" dataset for the rows that are "main". Note that the values for "aux" rows will be NaN (50%).

In [36]:
# same drop list you used when training
drop_cols = ["own_value_ls", "D", "fold"]

# main fold
main = full[full["fold"] == "main"].copy()

# features for main (must match training: drop non-features)
X_main = main.drop(columns=drop_cols).copy()


# predict μ̂1 and μ̂0 on main, then S = μ̂1 − μ̂0
model_u1_hat = model_u1.predict(X_main)
model_u0_hat = model_u0.predict(X_main)

# we filter full by the location of the X_main, and then create a new column
full.loc[main.index, "S_xgb"] = model_u1_hat - model_u0_hat



In [37]:
# quick sanity check
print(full.loc[main.index, "S_xgb"].describe())

count      864.000000
mean      3393.567139
std       8601.126953
min     -24875.205078
25%       -285.228180
50%        839.805725
75%       3532.338196
max      55522.324219
Name: S_xgb, dtype: float64


In [38]:
full.head()

,two_project_fulla_1,ls_full,ls_rest,ls_half,ls_poultry_full,ls_poultry_rest,ls_poultry_half,male_designee_full,male_designee_half,treated_HHs,share_all,precovid,own_ls,small_ls_bl,medium_ls_bl,large_ls_bl,w05_ls_value,fattening_ls_count,sold_ls,sold_ls_count,consumed_ls,consumed_ls_count,ls_stable,dwelling_type_bin_1,dwelling_type_bin_2,dwelling_type_bin_3,dwelling_type_bin_4,dwelling_type_bin_5,dwelling_type_bin_6,dwelling_ownership_b,dwelling_ownershi_2,dwelling_rent,tertile_dwellperpers,tertile_dwellperp_3,tertile_dwellperp_4,water_source_dist,hh_share_latrine,latrine,water_source_bin_2,water_source_bin_3,water_heating_source,water_heating_sou_5,water_heating_sou_6,water_heating_sou_7,water_heating_sou_8,water_heating_sou_9,sewage_e_bin_1,sewage_e_bin_2,sewage_e_bin_3,sewage_e_bin_4,sewage_e_bin_5,sewage_e_bin_6,ceiling_e_bin_1,ceiling_e_bin_2,ceiling_e_bin_3,ceiling_e_bin_4,ceiling_e_bin_5,ceiling_e_bin_6,ceiling_e_bin_7,ceiling_e_bin_9,flooring_e_bin_1,flooring_e_bin_2,flooring_e_bin_3,flooring_e_bin_4,flooring_e_bin_5,lighting_e_bin_1,lighting_e_bin_2,lighting_e_bin_4,home_improve_src__10,home_improve_src__11,home_improve_src__12,home_improve_src__13,home_improve_src__14,home_improve_src__15,home_improve_amount,ratio_smart_mobil_16,ratio_skiptimes_p_17,worry_about_food,got_medical_assist,medical_assist_place,medical_assist_pl_18,medical_assist_pl_19,medical_assist_pl_20,medical_assist_money,medical_assist_mo_21,medical_assist_mo_22,medical_assist_mo_23,ladder,ratio_eatoutside__24,non_food_imputed__25,nonfood_exp_mon_bl,food_exp_bl,nonagri_grp_proce_26,nonagri_grp_buy_sell,nonagri_grp_services,nonagri_atleast1_loc,nonagri_atleast1__27,nonagri_sumcapital,agri_own_yesno,agri_own_size_quir,agri_rent_yesno,agri_rent_size_quir,agri_rent_type_bin_2,agri_rent_type_bin_3,agri_rent_amount_bl,agri_rent_out_yesno,agri_rent_out_siz_28,agri_rent_out_type_b,agri_rent_out_typ_29,agri_rent_out_amount,agri_l_assets_cart,agri_l_assets_shovel,agri_l_assets_plow,agri_l_assets_sickle,agri_l_assets_other,agri_l_assets_non,hhmembers_agri_work,bl_fruit_trees_count,totalincome_witho_32,totalincome_1mo,femhd,oldhd,younghd,singhd,illithd,haschld,propdisabled_ill,hhdisabl_chld,hhdisabl_adult,can_start_busines_33,worked_with_busin_34,use_time_forrev_atl,use_time_rev_30da_35,use_time_foredu_atl,use_time_edu_30da_36,nobank_r_too_far_atl,nobank_r_too_expe_37,nobank_r_lack_of__38,nobank_r_no_trust_39,nobank_r_religiou_40,nobank_r_not_enou_41,nobank_r_some1_el_42,nobank_r_no_need_atl,rosca_r_nonfarm_e_43,rosca_r_for_marri_44,rosca_r_for_educa_45,rosca_r_buy_a_hou_46,rosca_r_medical_e_47,rosca_r_loss_of_a_48,rosca_r_other_atl,help_mother_bin_2_49,help_mother_bin_3_50,help_father_bin_2_51,help_father_bin_3_52,help_siblings_bin_53,help_siblings_bin_54,help_hhmembers_outsi,help_hhmembers_ou_55,help_friends_bin__56,help_friends_bin__57,help_others_bin_2_58,help_others_bin_3_59,health_cant_work__60,health_cant_work__61,dm_food_atleast,dm_cloth_atleast,dm_educ_atleast,dm_health_atleast,dm_home_atleast,dm_food_influence_bi,dm_food_influence_62,dm_food_influence_63,dm_cloth_influence_b,dm_cloth_influenc_64,dm_cloth_influenc_65,dm_educ_influence_bi,dm_educ_influence_66,dm_educ_influence_67,dm_health_influence_,dm_health_influen_68,dm_health_influen_69,dm_home_influence_bi,dm_home_influence_70,dm_home_influence_71,dm_money_earn_bin_72,dm_money_earn_bin_73,dm_money_earn_bin_74,dm_money_earn_bin_75,dm_spouse_earn_bi_76,dm_spouse_earn_bi_77,dm_spouse_earn_bi_78,dm_spouse_earn_bi_79,dm_daily_spend_bi_80,dm_daily_spend_bi_81,dm_daily_spend_bi_82,dm_daily_spend_bi_83,dm_major_spend_bi_84,dm_major_spend_bi_85,dm_major_spend_bi_86,dm_major_spend_bi_87,dm_fplan_child_bi_88,dm_fplan_child_bi_89,dm_fplan_child_bi_90,dm_fplan_child_bi_91,dm_child_educ_bin_92,dm_child_educ_bin_93,dm_child_educ_bin_94,dm_child_educ_bin_95,dm_business_bin_1_96,dm_business_bin_2_97,dm_business_bin_3_98,dm_business_bin_4_99,health_rank_bin_1_100,health_rank_bin_2_101,health_rank_bin_3_102,health_ra

---


## BLP

$$
Y_i \;=\; \alpha \;+\; \beta_1\, (D_i - p) \;+\; \beta_2\, (D_i - p)\,\tilde{S}_i \;+\;X_{1i}^\top \delta \; + \varepsilon_i
$$

1. We need to shift the ml proxy score so its mean (on the main fold) is zero. create a new column "S_xgb_shifted"
2. We also need to calculate D-p(Z) for each row in the main fold.
3. Combing the two new columns and create a single (D-p)*shifted_S_xgb column

In [39]:
# Creating the D-p column:
mask = full['fold'].eq('main')
full['D-p'] = np.nan
full.loc[mask, 'D-p'] = full.loc[mask, 'D'].astype(float) - (1.0/3.0)

# Create the shifted (centered) ML proxy on the MAIN sample
mask = full['fold'].eq('main')
full['S_xgb_shifted'] = np.nan
mu_S = full.loc[mask, 'S_xgb'].mean()
full.loc[mask, 'S_xgb_shifted'] = full.loc[mask, 'S_xgb'] - mu_S

# Creating a single (D-p)*shifted_S_xgb column
mask = full['fold'].eq('main')
full['D-p_S-shifted'] = np.nan
full.loc[mask, 'D-p_S-shifted'] = full.loc[mask, 'D-p'].astype(float) * full.loc[mask, 'S_xgb_shifted'].astype(float)


In [40]:
full.head(3)

,two_project_fulla_1,ls_full,ls_rest,ls_half,ls_poultry_full,ls_poultry_rest,ls_poultry_half,male_designee_full,male_designee_half,treated_HHs,share_all,precovid,own_ls,small_ls_bl,medium_ls_bl,large_ls_bl,w05_ls_value,fattening_ls_count,sold_ls,sold_ls_count,consumed_ls,consumed_ls_count,ls_stable,dwelling_type_bin_1,dwelling_type_bin_2,dwelling_type_bin_3,dwelling_type_bin_4,dwelling_type_bin_5,dwelling_type_bin_6,dwelling_ownership_b,dwelling_ownershi_2,dwelling_rent,tertile_dwellperpers,tertile_dwellperp_3,tertile_dwellperp_4,water_source_dist,hh_share_latrine,latrine,water_source_bin_2,water_source_bin_3,water_heating_source,water_heating_sou_5,water_heating_sou_6,water_heating_sou_7,water_heating_sou_8,water_heating_sou_9,sewage_e_bin_1,sewage_e_bin_2,sewage_e_bin_3,sewage_e_bin_4,sewage_e_bin_5,sewage_e_bin_6,ceiling_e_bin_1,ceiling_e_bin_2,ceiling_e_bin_3,ceiling_e_bin_4,ceiling_e_bin_5,ceiling_e_bin_6,ceiling_e_bin_7,ceiling_e_bin_9,flooring_e_bin_1,flooring_e_bin_2,flooring_e_bin_3,flooring_e_bin_4,flooring_e_bin_5,lighting_e_bin_1,lighting_e_bin_2,lighting_e_bin_4,home_improve_src__10,home_improve_src__11,home_improve_src__12,home_improve_src__13,home_improve_src__14,home_improve_src__15,home_improve_amount,ratio_smart_mobil_16,ratio_skiptimes_p_17,worry_about_food,got_medical_assist,medical_assist_place,medical_assist_pl_18,medical_assist_pl_19,medical_assist_pl_20,medical_assist_money,medical_assist_mo_21,medical_assist_mo_22,medical_assist_mo_23,ladder,ratio_eatoutside__24,non_food_imputed__25,nonfood_exp_mon_bl,food_exp_bl,nonagri_grp_proce_26,nonagri_grp_buy_sell,nonagri_grp_services,nonagri_atleast1_loc,nonagri_atleast1__27,nonagri_sumcapital,agri_own_yesno,agri_own_size_quir,agri_rent_yesno,agri_rent_size_quir,agri_rent_type_bin_2,agri_rent_type_bin_3,agri_rent_amount_bl,agri_rent_out_yesno,agri_rent_out_siz_28,agri_rent_out_type_b,agri_rent_out_typ_29,agri_rent_out_amount,agri_l_assets_cart,agri_l_assets_shovel,agri_l_assets_plow,agri_l_assets_sickle,agri_l_assets_other,agri_l_assets_non,hhmembers_agri_work,bl_fruit_trees_count,totalincome_witho_32,totalincome_1mo,femhd,oldhd,younghd,singhd,illithd,haschld,propdisabled_ill,hhdisabl_chld,hhdisabl_adult,can_start_busines_33,worked_with_busin_34,use_time_forrev_atl,use_time_rev_30da_35,use_time_foredu_atl,use_time_edu_30da_36,nobank_r_too_far_atl,nobank_r_too_expe_37,nobank_r_lack_of__38,nobank_r_no_trust_39,nobank_r_religiou_40,nobank_r_not_enou_41,nobank_r_some1_el_42,nobank_r_no_need_atl,rosca_r_nonfarm_e_43,rosca_r_for_marri_44,rosca_r_for_educa_45,rosca_r_buy_a_hou_46,rosca_r_medical_e_47,rosca_r_loss_of_a_48,rosca_r_other_atl,help_mother_bin_2_49,help_mother_bin_3_50,help_father_bin_2_51,help_father_bin_3_52,help_siblings_bin_53,help_siblings_bin_54,help_hhmembers_outsi,help_hhmembers_ou_55,help_friends_bin__56,help_friends_bin__57,help_others_bin_2_58,help_others_bin_3_59,health_cant_work__60,health_cant_work__61,dm_food_atleast,dm_cloth_atleast,dm_educ_atleast,dm_health_atleast,dm_home_atleast,dm_food_influence_bi,dm_food_influence_62,dm_food_influence_63,dm_cloth_influence_b,dm_cloth_influenc_64,dm_cloth_influenc_65,dm_educ_influence_bi,dm_educ_influence_66,dm_educ_influence_67,dm_health_influence_,dm_health_influen_68,dm_health_influen_69,dm_home_influence_bi,dm_home_influence_70,dm_home_influence_71,dm_money_earn_bin_72,dm_money_earn_bin_73,dm_money_earn_bin_74,dm_money_earn_bin_75,dm_spouse_earn_bi_76,dm_spouse_earn_bi_77,dm_spouse_earn_bi_78,dm_spouse_earn_bi_79,dm_daily_spend_bi_80,dm_daily_spend_bi_81,dm_daily_spend_bi_82,dm_daily_spend_bi_83,dm_major_spend_bi_84,dm_major_spend_bi_85,dm_major_spend_bi_86,dm_major_spend_bi_87,dm_fplan_child_bi_88,dm_fplan_child_bi_89,dm_fplan_child_bi_90,dm_fplan_child_bi_91,dm_child_educ_bin_92,dm_child_educ_bin_93,dm_child_educ_bin_94,dm_child_educ_bin_95,dm_business_bin_1_96,dm_business_bin_2_97,dm_business_bin_3_98,dm_business_bin_4_99,health_rank_bin_1_100,health_rank_bin_2_101,health_rank_bin_3_102,health_ra

The BLP require some additions, they are used for noise reductions

In [41]:
# 1) Baseline proxy B_i (The control predictions column)

# model_u0 was trained on the auxiliary data
# X_main: feature matrix for rows where full['fold'] == 'main', with identical preprocessing

model_u0_hat = model_u0.predict(X_main)

mask = full['fold'].eq('main')
full.loc[mask, 'B_proxy'] = pd.Series(model_u0_hat, index=full.index[mask])


# 2) p * S (use raw S_xgb, not centered)
full['p_times_S'] = np.nan
full.loc[mask, 'p_times_S'] = (1.0/3.0) * full.loc[mask, 'S_xgb']


## Fitting the BLP model

In [42]:
import statsmodels.api as sm

mask = full['fold'].eq('main')

y = full.loc[mask, 'own_value_ls']  
X = full.loc[mask, ['D-p', 'D-p_S-shifted', 'B_proxy', 'p_times_S']]
#X = full.loc[mask, ['D-p', 'D-p_S-shifted']] # Without those noise reduction 

X = sm.add_constant(X)  # adds the intercept α

blp = sm.OLS(y, X).fit(cov_type='HC1')  # robust SEs

print(blp.summary())

# Mapping to BLP notation:
#   coef on 'D-p'            -> β1  (average treatment effect)
#   coef on 'D-p_S-shifted'  -> β2  (heterogeneity captured by S)


                            OLS Regression Results                            
Dep. Variable:           own_value_ls   R-squared:                       0.578
Model:                            OLS   Adj. R-squared:                  0.576
Method:                 Least Squares   F-statistic:                     58.16
Date:                Fri, 19 Sep 2025   Prob (F-statistic):           1.83e-43
Time:                        11:47:40   Log-Likelihood:                -9380.7
No. Observations:                 864   AIC:                         1.877e+04
Df Residuals:                     859   BIC:                         1.880e+04
Df Model:                           4                                         
Covariance Type:                  HC1                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const          -411.6772    265.701     -1.549

In [43]:
# column keys must match exactly how you named them in X
k1 = "D-p"
k2 = "D-p_S-shifted"

out = {
    "r2": blp.rsquared,
    "r2_adj": blp.rsquared_adj,
    "D_minus_p": {},
    "D_minus_p_times_S": {}
}

def grab(key):
    ci_low, ci_high = blp.conf_int().loc[key]  # 95% CI by default
    return {
        "coef":    blp.params[key],
        "std_err": blp.bse[key],
        "z":       blp.tvalues[key],   # z-values since cov_type='HC1'
        "p":       blp.pvalues[key],
        "ci_low":  ci_low,
        "ci_high": ci_high,
    }

out["D_minus_p"] = grab(k1)
out["D_minus_p_times_S"] = grab(k2)

results = {"blp": []}
results["blp"].append(out)


In [44]:
results 

{'blp': [{'r2': np.float64(0.5781146622705222),
   'r2_adj': np.float64(0.5761501205348785),
   'D_minus_p': {'coef': np.float64(2356.1204487519562),
    'std_err': np.float64(934.3247286568846),
    'z': np.float64(2.5217361549864346),
    'p': np.float64(0.011677726141719214),
    'ci_low': 524.8776307193039,
    'ci_high': 4187.363266784609},
   'D_minus_p_times_S': {'coef': np.float64(0.8562569860805366),
    'std_err': np.float64(0.2313698499676872),
    'z': np.float64(3.700814890964057),
    'p': np.float64(0.0002149082104492074),
    'ci_low': 0.4027804130354339,
    'ci_high': 1.3097335591256392}}]}

In [45]:
results['blp'][0]['D_minus_p']

{'coef': np.float64(2356.1204487519562),
 'std_err': np.float64(934.3247286568846),
 'z': np.float64(2.5217361549864346),
 'p': np.float64(0.011677726141719214),
 'ci_low': 524.8776307193039,
 'ci_high': 4187.363266784609}

---


## GATES

### Doing the GATES Regression

$$
Y_i \;=\; \alpha \;+\; \sum_{k=1}^{5} \gamma_k\,(D_i - p)\,\mathbf{1}\{\text{Group}_i = k\}
\;+\; X_{1i}^\top \delta \;+\; \varepsilon_i
$$

 
- With $T_k = (D-p)\cdot \mathbf{1}\{\text{Group}=k\}$, the regression is $Y = \sum_{k=1}^5 \gamma_k T_k + u$.
- In the code we will create five columns $T_k$ for each group $k$ and run the regression on them.

##### First assigning each ML proxy (in main) to a one of K=5 groups based on the quantiles of the proxy

In [46]:
# compute quintile cutoffs on MAIN only
mask_main = full["fold"].eq("main")
s_main = full.loc[mask_main, "S_xgb"].dropna()

probs   = [0, .2, .4, .6, .8, 1.0]
cutoffs = s_main.quantile(probs, interpolation="linear").to_numpy()

# assign groups on MAIN only
full["Group"] = np.nan

full.loc[mask_main, "Group"] = pd.cut(
    full.loc[mask_main, "S_xgb"],
    bins=cutoffs,
    labels=[1, 2, 3, 4, 5],
    include_lowest=True,
    right=True
).astype("float")



In [47]:
full

,two_project_fulla_1,ls_full,ls_rest,ls_half,ls_poultry_full,ls_poultry_rest,ls_poultry_half,male_designee_full,male_designee_half,treated_HHs,share_all,precovid,own_ls,small_ls_bl,medium_ls_bl,large_ls_bl,w05_ls_value,fattening_ls_count,sold_ls,sold_ls_count,consumed_ls,consumed_ls_count,ls_stable,dwelling_type_bin_1,dwelling_type_bin_2,dwelling_type_bin_3,dwelling_type_bin_4,dwelling_type_bin_5,dwelling_type_bin_6,dwelling_ownership_b,dwelling_ownershi_2,dwelling_rent,tertile_dwellperpers,tertile_dwellperp_3,tertile_dwellperp_4,water_source_dist,hh_share_latrine,latrine,water_source_bin_2,water_source_bin_3,water_heating_source,water_heating_sou_5,water_heating_sou_6,water_heating_sou_7,water_heating_sou_8,water_heating_sou_9,sewage_e_bin_1,sewage_e_bin_2,sewage_e_bin_3,sewage_e_bin_4,sewage_e_bin_5,sewage_e_bin_6,ceiling_e_bin_1,ceiling_e_bin_2,ceiling_e_bin_3,ceiling_e_bin_4,ceiling_e_bin_5,ceiling_e_bin_6,ceiling_e_bin_7,ceiling_e_bin_9,flooring_e_bin_1,flooring_e_bin_2,flooring_e_bin_3,flooring_e_bin_4,flooring_e_bin_5,lighting_e_bin_1,lighting_e_bin_2,lighting_e_bin_4,home_improve_src__10,home_improve_src__11,home_improve_src__12,home_improve_src__13,home_improve_src__14,home_improve_src__15,home_improve_amount,ratio_smart_mobil_16,ratio_skiptimes_p_17,worry_about_food,got_medical_assist,medical_assist_place,medical_assist_pl_18,medical_assist_pl_19,medical_assist_pl_20,medical_assist_money,medical_assist_mo_21,medical_assist_mo_22,medical_assist_mo_23,ladder,ratio_eatoutside__24,non_food_imputed__25,nonfood_exp_mon_bl,food_exp_bl,nonagri_grp_proce_26,nonagri_grp_buy_sell,nonagri_grp_services,nonagri_atleast1_loc,nonagri_atleast1__27,nonagri_sumcapital,agri_own_yesno,agri_own_size_quir,agri_rent_yesno,agri_rent_size_quir,agri_rent_type_bin_2,agri_rent_type_bin_3,agri_rent_amount_bl,agri_rent_out_yesno,agri_rent_out_siz_28,agri_rent_out_type_b,agri_rent_out_typ_29,agri_rent_out_amount,agri_l_assets_cart,agri_l_assets_shovel,agri_l_assets_plow,agri_l_assets_sickle,agri_l_assets_other,agri_l_assets_non,hhmembers_agri_work,bl_fruit_trees_count,totalincome_witho_32,totalincome_1mo,femhd,oldhd,younghd,singhd,illithd,haschld,propdisabled_ill,hhdisabl_chld,hhdisabl_adult,can_start_busines_33,worked_with_busin_34,use_time_forrev_atl,use_time_rev_30da_35,use_time_foredu_atl,use_time_edu_30da_36,nobank_r_too_far_atl,nobank_r_too_expe_37,nobank_r_lack_of__38,nobank_r_no_trust_39,nobank_r_religiou_40,nobank_r_not_enou_41,nobank_r_some1_el_42,nobank_r_no_need_atl,rosca_r_nonfarm_e_43,rosca_r_for_marri_44,rosca_r_for_educa_45,rosca_r_buy_a_hou_46,rosca_r_medical_e_47,rosca_r_loss_of_a_48,rosca_r_other_atl,help_mother_bin_2_49,help_mother_bin_3_50,help_father_bin_2_51,help_father_bin_3_52,help_siblings_bin_53,help_siblings_bin_54,help_hhmembers_outsi,help_hhmembers_ou_55,help_friends_bin__56,help_friends_bin__57,help_others_bin_2_58,help_others_bin_3_59,health_cant_work__60,health_cant_work__61,dm_food_atleast,dm_cloth_atleast,dm_educ_atleast,dm_health_atleast,dm_home_atleast,dm_food_influence_bi,dm_food_influence_62,dm_food_influence_63,dm_cloth_influence_b,dm_cloth_influenc_64,dm_cloth_influenc_65,dm_educ_influence_bi,dm_educ_influence_66,dm_educ_influence_67,dm_health_influence_,dm_health_influen_68,dm_health_influen_69,dm_home_influence_bi,dm_home_influence_70,dm_home_influence_71,dm_money_earn_bin_72,dm_money_earn_bin_73,dm_money_earn_bin_74,dm_money_earn_bin_75,dm_spouse_earn_bi_76,dm_spouse_earn_bi_77,dm_spouse_earn_bi_78,dm_spouse_earn_bi_79,dm_daily_spend_bi_80,dm_daily_spend_bi_81,dm_daily_spend_bi_82,dm_daily_spend_bi_83,dm_major_spend_bi_84,dm_major_spend_bi_85,dm_major_spend_bi_86,dm_major_spend_bi_87,dm_fplan_child_bi_88,dm_fplan_child_bi_89,dm_fplan_child_bi_90,dm_fplan_child_bi_91,dm_child_educ_bin_92,dm_child_educ_bin_93,dm_child_educ_bin_94,dm_child_educ_bin_95,dm_business_bin_1_96,dm_business_bin_2_97,dm_business_bin_3_98,dm_business_bin_4_99,health_rank_bin_1_100,health_rank_bin_2_101,health_rank_bin_3_102,health_ra

In [48]:
mask = full["fold"].eq("main")
for k in range(1, 6):
    full.loc[mask, f"T{k}"] = full.loc[mask, "D-p"] * (full.loc[mask, "Group"] == k).astype(int)

In [49]:
dfm = full.loc[(full["fold"]=="main") & full["Group"].notna(),
               ["own_value_ls","T1","T2","T3","T4","T5","B_proxy"]]

X = sm.add_constant(dfm[["T1","T2","T3","T4","T5","B_proxy"]], has_constant="add")
model = sm.OLS(dfm["own_value_ls"], X).fit()
model.params

const       -446.486234
T1         -3803.454541
T2            46.470885
T3           298.339236
T4           123.390425
T5         14826.925173
B_proxy        1.162647
dtype: float64

- `T1` … `T5` are the **GATES group treatment effects** (the $\gamma_k$’s). Each $\gamma_k$ is the estimated average impact of treatment on `own_value_ls` **within group $k$**, because we regressed $Y$ on $(D-p)\cdot \mathbf{1}\{\text{Group}=k\}$.  
  - The numbers:  
    - Group 1: ~**−5.1k**  
    - Group 2: ~**−177**  
    - Group 3: ~**+548**  
    - Group 4: ~**+2.6k**  
    - Group 5: ~**+13.8k**  
  These are in the **units of `own_value_ls`** and tells how much treatment changes the outcome for each quintile of the proxy.

- **`B_proxy`** is an **$X_1$ control** (baseline predictor of the outcome).



In [50]:
res   = model.get_robustcov_results(cov_type="HC1")  
names = model.model.exog_names                       

params = pd.Series(res.params, index=names, name="gamma")
bse    = pd.Series(res.bse,    index=names, name="se")
ci     = pd.DataFrame(res.conf_int(), index=names, columns=["ci_lo","ci_hi"])

gates = pd.concat([params, bse, ci], axis=1).loc[["T1","T2","T3","T4","T5"]]
gates.index = [1,2,3,4,5]  # label by group (optional)
gates


,gamma,se,ci_lo,ci_hi
1,-3803.454541,2408.623772,-8530.946991,924.037909
2,46.470885,339.099514,-619.091920,712.033690
3,298.339236,370.915621,-429.670185,1026.348657
4,123.390425,744.573214,-1338.010184,1584.791034
5,14826.925173,3570.782068,7818.422865,21835.427482


---

# **CLAN**

**First, we will get a list of column names with only the features that will be used in CLAN**
1. Get only the numeric columns.
2. Remove all the columns created in the previous steps.
3. Remove all the columns that numericly coded but are binary.
4. Remove all the columns that are constant (have one unique value).

In [51]:
# 1. Keeping only numeric columns
full_numeric = full.select_dtypes(include=[np.number])

# 2. Removing the columns created in the previous steps
clan_exclude = [
    "D",
    "S_xgb",
    "D-p",
    "S_xgb_shifted",
    "D-p_S-shifted",
    "B_proxy",
    "p_times_S",
    "Group",
    "T1",
    "T2",
    "T3",
    "T4",
    "T5",
]
# 3. Removing binary columns:
binary_cols_01 = [c for c in full_numeric.columns
                  if set(full_numeric[c].dropna().unique()) == {0, 1}]


# 4) Constant numeric columns (exactly one unique value)
#const_cols = [c for c in full_numeric.columns
 #             if full_numeric[c].nunique(dropna=True) == 1]

# Combing all exclude column lists and filter the numeric DF:
cols_to_remove = clan_exclude + binary_cols_01

# filtering the numeric DF
full_clan_cols = full_numeric.drop(columns=cols_to_remove)

# This is our final feature list
clan_cols = full_clan_cols.columns

print(len(clan_cols))

47


#### **Creating the CLAN table:**
1. Filter to inlcude only the main fold
2. Include only the CLAN columns + the group column
3. Include only group 1 and 5
4. Calculate the mean of the CLAN columns for each group

In [52]:
# filtering to include only main fold
full_main = full[full['fold'] == 'main']

# Filtering to include only CLAN columns + group
full_clan = full_main.loc[:, list(clan_cols) + ["Group"]].copy()

# keep only groups 1 and 5 (fixing the var name)
full_clan = full_clan[full_clan["Group"].isin([1, 5])]

# means by group, then transpose so rows are covariates
means = full_clan.groupby("Group")[clan_cols].mean().reindex([1, 5])
clan_means = means.T
clan_means.columns = ["Mean G1 (predicted least affected)", "Mean G5 (predicted most affected)"]

clan_means.head()

,Mean G1 (predicted least affected),Mean G5 (predicted most affected)
treated_HHs,7.757226,11.202312
share_all,31.051523,47.516132
small_ls_bl,3.482659,4.427746
medium_ls_bl,0.213873,0.317919
large_ls_bl,0.248555,0.202312


In [53]:
clan_means

,Mean G1 (predicted least affected),Mean G5 (predicted most affected)
treated_HHs,7.757226,11.202312
share_all,31.051523,47.516132
small_ls_bl,3.482659,4.427746
medium_ls_bl,0.213873,0.317919
large_ls_bl,0.248555,0.202312
w05_ls_value,1442.456055,1776.21167
fattening_ls_count,0.32948,0.468208
sold_ls_count,0.156069,0.190751
consumed_ls_count,1.184971,1.49711
dwelling_rent,2.307101,5.174153


## Standard errors for CLAN
**Where:**  
- $x_{ki}$ = value for unit $i$ in group $k$  
- $n_k$ = number of units in group $k$  
- $\bar{x}_k$ = sample mean in group $k$  
- $s_k^2$ = sample variance in group $k$ (with $s_k=\sqrt{s_k^2}$)  
- $s_p^2$ = pooled variance (only if equal variances are assumed)  
- $\widehat{\Delta}$ = difference in means (Group 5 − Group 1)  
- $z$ = test statistic (estimate divided by its SE)  
- $z_{1-\alpha/2}$ = standard normal critical value (e.g., $1.96$ for $95\%$ CI)  
- $\Phi(\cdot)$ = standard normal CDF


**SE of a group mean**
$$
\mathrm{SE}(\bar{x}_k) \;=\; \frac{s_k}{\sqrt{n_k}}
$$
Brief: uncertainty of $\bar{x}_k$; shrinks with $\sqrt{n_k}$.

**Difference in means (G5 − G1)**
$$
\widehat{\Delta} \;=\; \bar{x}_5 \;-\; \bar{x}_1
$$
Brief: contrast between most- and least-affected groups.

**SE of the difference (Welch, unequal variances — default)**
$$
\mathrm{SE}(\widehat{\Delta}) \;=\; \sqrt{\frac{s_5^2}{n_5} \;+\; \frac{s_1^2}{n_1}}
$$
Brief: uses each group’s own variance; no equal-variance assumption.

**z-statistic, CI, and two-sided p-value (normal reference)**
$$
z \;=\; \frac{\text{estimate}}{\mathrm{SE}}, 
\qquad
\text{CI}_{1-\alpha}:\;\; \text{estimate} \;\pm\; z_{1-\alpha/2}\cdot \mathrm{SE},
\qquad
p \;=\; 2\bigl(1-\Phi(|z|)\bigr).
$$
Brief: test and interval built from the estimate’s SE.


In [54]:
from math import erf, sqrt

# Per-group stats (mean, sd, n) for each CLAN variable
stats = (
    full_clan.groupby("Group")[clan_cols]
    .agg(["mean", "std", "count"])
    .reindex([1, 5])  # ensure order: G1, G5
)

rows = []
zcrit = 1.96  # 95% CI using normal critical value

for col in clan_cols:
    m1 = stats.loc[1, (col, "mean")]
    m5 = stats.loc[5, (col, "mean")]
    s1 = stats.loc[1, (col, "std")]
    s5 = stats.loc[5, (col, "std")]
    n1 = stats.loc[1, (col, "count")]
    n5 = stats.loc[5, (col, "count")]

    diff = m5 - m1
    
    # Welch SE for difference in means
    se = np.sqrt((s5**2) / n5 + (s1**2) / n1) 
    z  = diff / se

    # Two-sided p-value from the normal CDF via erf
    # Phi(z) = 0.5 * (1 + erf(z / sqrt(2)))
    
    p_two_sided = 2 * (1 - (0.5 * (1 + erf(abs(z) / sqrt(2)))))
   

    ci_lo = diff - zcrit * se 
    ci_hi = diff + zcrit * se 

    rows.append({
        "covariate": col,
        "Mean G1 (predicted least affected)": m1,
        "Mean G5 (predicted most affected)": m5,
        "Diff (G5 - G1)": diff,
        "SE (Welch)": se,
        "z": z,
        "p (two-sided)": p_two_sided,
        "CI 95% lower": ci_lo,
        "CI 95% upper": ci_hi,
    })

clan_table = pd.DataFrame(rows).set_index("covariate")
clan_table.head()


C:\Users\ahmed\AppData\Local\Temp\ipykernel_22692\3455458517.py:25: RuntimeWarning: invalid value encountered in scalar divide
  z  = diff / se


,Mean G1 (predicted least affected),Mean G5 (predicted most affected),Diff (G5 - G1),SE (Welch),z,p (two-sided),CI 95% lower,CI 95% upper
covariate,,,,,,,,
treated_HHs,7.757226,11.202312,3.445087,0.827746,4.162010,3.154589e-05,1.822705,5.067469
share_all,31.051523,47.516132,16.464609,2.437065,6.755916,1.419354e-11,11.687961,21.241257
small_ls_bl,3.482659,4.427746,0.945087,0.440192,2.146987,3.179431e-02,0.082310,1.807864
medium_ls_bl,0.213873,0.317919,0.104046,0.089547,1.161923,2.452667e-01,-0.071465,0.279558
large_ls_bl,0.248555,0.202312,-0.046243,0.068158,-0.678466,4.974763e-01,-0.179832,0.087347


In [55]:
clan_table

,Mean G1 (predicted least affected),Mean G5 (predicted most affected),Diff (G5 - G1),SE (Welch),z,p (two-sided),CI 95% lower,CI 95% upper
covariate,,,,,,,,
treated_HHs,7.757226,11.202312,3.445087,0.827746,4.162010,3.154589e-05,1.822705,5.067469
share_all,31.051523,47.516132,16.464609,2.437065,6.755916,1.419354e-11,11.687961,21.241257
small_ls_bl,3.482659,4.427746,0.945087,0.440192,2.146987,3.179431e-02,0.082310,1.807864
medium_ls_bl,0.213873,0.317919,0.104046,0.089547,1.161923,2.452667e-01,-0.071465,0.279558
large_ls_bl,0.248555,0.202312,-0.046243,0.068158,-0.678466,4.974763e-01,-0.179832,0.087347
w05_ls_value,1442.456055,1776.211670,333.755615,338.977483,0.984595,3.248229e-01,-330.640252,998.151482
fattening_ls_count,0.329480,0.468208,0.138728,0.238550,0.581549,5.608706e-01,-0.328829,0.606286
sold_ls_count,0.156069,0.190751,0.034682,0.102314,0.338976,7.346275e-01,-0.165854,0.235218
consumed_ls_count,1.184971,1.497110,0.312139,0.323567,0.964679,3.347056e-01,-0.322053,0.946331


In [56]:
clan_table[clan_table['p (two-sided)'] < 0.05]

,Mean G1 (predicted least affected),Mean G5 (predicted most affected),Diff (G5 - G1),SE (Welch),z,p (two-sided),CI 95% lower,CI 95% upper
covariate,,,,,,,,
treated_HHs,7.757226,11.202312,3.445087,0.827746,4.162010,3.154589e-05,1.822705,5.067469
share_all,31.051523,47.516132,16.464609,2.437065,6.755916,1.419354e-11,11.687961,21.241257
small_ls_bl,3.482659,4.427746,0.945087,0.440192,2.146987,3.179431e-02,0.082310,1.807864
home_improve_amount,4116.416185,801.358382,-3315.057803,1561.142849,-2.123481,3.371353e-02,-6374.897788,-255.217819
food_exp_bl,337.280640,290.617340,-46.663300,16.340580,-2.855670,4.294615e-03,-78.690836,-14.635764
bl_fruit_trees_count,0.023121,0.190751,0.167630,0.067758,2.473965,1.336227e-02,0.034825,0.300435
totalincome_1mo,886.803467,1738.189819,851.386353,168.512727,5.052356,4.363924e-07,521.101408,1181.671297
propdisabled_ill,0.189097,0.133816,-0.055281,0.020890,-2.646245,8.139078e-03,-0.096226,-0.014336
health_cant_work__61,4.855491,2.791908,-2.063584,0.642302,-3.212793,1.314510e-03,-3.322496,-0.804672


In [57]:
clan_table.shape[0]

47

outcome_list <- c("totalincome_1mo", "elset_sh_eq", "own_value_ls", "tot_exp_2")